# Tutorial 05: Problem Variations and Parameter Studies

Learn how to vary problem parameters and explore the solution space.

## Learning Objectives

By the end of this tutorial, you will understand:
- How to create problem variations by changing parameters
- How to compare solutions across different settings
- How to analyze MFG solution outputs
- Next steps for advanced usage

This tutorial wraps up the series and points you toward more advanced topics.

**Time estimate**: 15 minutes

## Step 1: Import Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from mfg_pde import MFGProblem
from mfg_pde.core import MFGComponents
from mfg_pde.core.hamiltonian import QuadraticControlCost, SeparableHamiltonian
from mfg_pde.geometry import TensorProductGrid
from mfg_pde.geometry.boundary import no_flux_bc

## Helper: Create Standard LQ Components

We'll define a helper function to create Linear-Quadratic MFG components with adjustable coupling strength.

In [ ]:
def create_lq_components(coupling_strength: float = 0.5):
    """
    Create standard Linear-Quadratic MFG components.

    This is the default setup used throughout this tutorial:
    - Hamiltonian: H(p,m) = (1/2)|p|^2 + coupling * m
    - Terminal cost: (x - 0.5)^2 (agents want to be at center)
    - Initial density: Gaussian at center
    """
    hamiltonian = SeparableHamiltonian(
        control_cost=QuadraticControlCost(control_cost=1.0),
        coupling=lambda m: coupling_strength * m,
        coupling_dm=lambda m: coupling_strength,
    )

    return MFGComponents(
        hamiltonian=hamiltonian,
        m_initial=lambda x: np.exp(-50 * (x - 0.5) ** 2),
        u_final=lambda x: (x - 0.5) ** 2,
    )


print("=" * 70)
print("TUTORIAL 05: Problem Variations and Parameter Studies")
print("=" * 70)
print()
print("We'll explore how changing parameters affects MFG solutions.")
print()

## Step 1: Diffusion Parameter Study

Varying diffusion changes how agents spread out over time.

In [ ]:
print("=" * 70)
print("DIFFUSION PARAMETER STUDY")
print("=" * 70)
print()

# Test different diffusion coefficients
diffusion_values = [0.05, 0.15, 0.30]
results_diffusion = {}

for diffusion in diffusion_values:
    print(f"Solving with diffusion={diffusion}...")
    geometry = TensorProductGrid(
        bounds=[(0.0, 1.0)],
        Nx_points=[50],
        boundary_conditions=no_flux_bc(dimension=1),
    )
    problem = MFGProblem(
        geometry=geometry,
        T=1.0,
        Nt=50,
        diffusion=diffusion,
        components=create_lq_components(coupling_strength=0.5),
    )
    results_diffusion[diffusion] = problem.solve(verbose=False)
    print(f"  Converged in {results_diffusion[diffusion].iterations} iterations")

print()

## Step 2: Coupling Strength Study

Varying coupling strength changes congestion effects.

In [ ]:
print("=" * 70)
print("COUPLING STRENGTH STUDY")
print("=" * 70)
print()

# Test different coupling strengths
coupling_values = [0.0, 0.5, 2.0]
results_coupling = {}

for coupling in coupling_values:
    print(f"Solving with coupling={coupling}...")
    geometry = TensorProductGrid(
        bounds=[(0.0, 1.0)],
        Nx_points=[50],
        boundary_conditions=no_flux_bc(dimension=1),
    )
    problem = MFGProblem(
        geometry=geometry,
        T=1.0,
        Nt=50,
        diffusion=0.15,
        components=create_lq_components(coupling_strength=coupling),
    )
    results_coupling[coupling] = problem.solve(verbose=False)
    print(f"  Converged in {results_coupling[coupling].iterations} iterations")

print()

## Step 3: Grid Resolution Study

Finer grids increase accuracy but also computation time.

In [ ]:
print("=" * 70)
print("GRID RESOLUTION STUDY")
print("=" * 70)
print()

# Test different resolutions
Nx_values = [25, 50, 100]
results_grid = {}

for Nx in Nx_values:
    print(f"Solving with Nx={Nx} grid points...")
    geometry = TensorProductGrid(
        bounds=[(0.0, 1.0)],
        Nx_points=[Nx],
        boundary_conditions=no_flux_bc(dimension=1),
    )
    problem = MFGProblem(
        geometry=geometry,
        T=1.0,
        Nt=Nx,  # Keep dt/dx ratio constant
        diffusion=0.15,
        components=create_lq_components(coupling_strength=0.5),
    )
    results_grid[Nx] = problem.solve(verbose=False)
    print(f"  Converged in {results_grid[Nx].iterations} iterations")

print()

## Step 4: Solution Analysis

Key outputs from any MFG solution.

In [ ]:
print("=" * 70)
print("SOLUTION ANALYSIS")
print("=" * 70)
print()

print("Key outputs from any MFG solution:")
print()
print("  Attribute         | Description")
print("  " + "-" * 50)
print("  result.U          | Value function u(t,x)")
print("  result.M          | Density m(t,x)")
print("  result.converged  | Convergence status")
print("  result.iterations | Number of iterations")
print("  result.max_error  | Final convergence error")
print()

# Mass conservation check
print("Mass Conservation Check (diffusion variations):")
for diffusion, result in results_diffusion.items():
    Nx = result.M.shape[1]
    dx = 1.0 / (Nx - 1)
    initial_mass = np.sum(result.M[0, :]) * dx
    final_mass = np.sum(result.M[-1, :]) * dx
    print(
        f"  diffusion={diffusion}: Initial={initial_mass:.4f}, "
        f"Final={final_mass:.4f}, Drift={abs(final_mass - initial_mass):.2e}"
    )

print()

In [ ]:
print("=" * 70)
print("PARAMETER EFFECTS SUMMARY")
print("=" * 70)
print()

print("Effect of parameters on MFG solutions:")
print()
print("  Parameter              | High Value Effect")
print("  " + "-" * 55)
print("  diffusion              | More spreading, smoother density")
print("  coupling_strength      | Stronger congestion avoidance")
print("  Nx (grid resolution)   | Higher accuracy, more computation")
print("  T (time horizon)       | More time for dynamics to evolve")
print()

## Step 5: Visualization

Compare solutions across all parameter studies.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Row 1: Diffusion study - final densities
ax = axes[0, 0]
for diffusion, result in results_diffusion.items():
    x_grid = np.linspace(0, 1, result.M.shape[1])
    ax.plot(x_grid, result.M[-1, :], label=f"diff={diffusion}")
ax.set_xlabel("x")
ax.set_ylabel("m(T, x)")
ax.set_title("Final Density: Diffusion Study")
ax.legend()
ax.grid(True, alpha=0.3)

# Row 1: Coupling study - final densities
ax = axes[0, 1]
for coupling, result in results_coupling.items():
    x_grid = np.linspace(0, 1, result.M.shape[1])
    ax.plot(x_grid, result.M[-1, :], label=f"coup={coupling}")
ax.set_xlabel("x")
ax.set_ylabel("m(T, x)")
ax.set_title("Final Density: Coupling Study")
ax.legend()
ax.grid(True, alpha=0.3)

# Row 1: Grid study - final densities
ax = axes[0, 2]
for Nx, result in results_grid.items():
    x_grid = np.linspace(0, 1, result.M.shape[1])
    ax.plot(x_grid, result.M[-1, :], label=f"Nx={Nx}")
ax.set_xlabel("x")
ax.set_ylabel("m(T, x)")
ax.set_title("Final Density: Resolution Study")
ax.legend()
ax.grid(True, alpha=0.3)

# Row 2: Value functions
ax = axes[1, 0]
for diffusion, result in results_diffusion.items():
    x_grid = np.linspace(0, 1, result.U.shape[1])
    ax.plot(x_grid, result.U[-1, :], label=f"diff={diffusion}")
ax.set_xlabel("x")
ax.set_ylabel("u(T, x)")
ax.set_title("Terminal Value: Diffusion Study")
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1, 1]
for coupling, result in results_coupling.items():
    x_grid = np.linspace(0, 1, result.U.shape[1])
    ax.plot(x_grid, result.U[-1, :], label=f"coup={coupling}")
ax.set_xlabel("x")
ax.set_ylabel("u(T, x)")
ax.set_title("Terminal Value: Coupling Study")
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1, 2]
for Nx, result in results_grid.items():
    x_grid = np.linspace(0, 1, result.U.shape[1])
    ax.plot(x_grid, result.U[-1, :], label=f"Nx={Nx}")
ax.set_xlabel("x")
ax.set_ylabel("u(T, x)")
ax.set_title("Terminal Value: Resolution Study")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

In [ ]:
print("=" * 70)
print("TUTORIAL COMPLETE")
print("=" * 70)
print()
print("What you learned:")
print("  1. Create parameter variations using helper functions")
print("  2. Compare diffusion, coupling, and resolution effects")
print("  3. Analyze mass conservation and convergence")
print("  4. Visualize parameter studies")
print()
print("Next steps:")
print("  - Explore examples/advanced/ for complex geometries")
print("  - See examples/applications/ for real-world problems")
print("  - Check docs/theory/ for mathematical background")
print()
print("=" * 70)